# Relu  
ReLU의 장점: 부분 활성화 가능, 기울기 소실 문제를 sigmoid보단 잘 잡음, 계산이 간단.

In [1]:
class Relu:
    def __init__(self):
        self.mask = None
    
    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0
        return out
    
    def backward(self, dout):
        dout[self.mask]=0
        dx=dout
        return dx

순전파의 입력인 x의 값이 0 이하인 인덱스는 True, 그 외는 False로

In [3]:
import numpy as np
x = np.array([[1.0, -0.5], [-2.0, 3.0]])
print(x)
mask = (x<=0)

print(mask)

[[ 1.  -0.5]
 [-2.   3. ]]
[[False  True]
 [ True False]]


순전파 때 입력 값이 0 이하먄 -> 역전파 때의 값은 0임 **

## Sigmoid  
Sigmoid의 문제점: 기울기소실문제, 업데이터 속도가 느림

In [4]:
class Sigmoid:
    def __init__(self):
        self.out = None
    
    def forward(self, x):
        out = 1/(1+np.exp(-x))
        self.out=out
        return out
    
    def backward(self, dout):
        dx = dout*(1.0-self.out)*self.out
        return dx

순전파의 출력을 out에 보관 했다가 역전파 계산 때 그 값을 사용

# Affine/Softmax 계층  

## Affine

신경망의 순전파 때 수행하는 행렬의 곱은 기하학에서는 어파인 변환이라고 해서 어파인 변환을 수행하는 처리를 'Affine 계층'이라고 명명

In [5]:
X = np.random.rand(2)
W = np.random.rand(2,3)
B = np.random.rand(3)

X.shape

(2,)

In [6]:
Y = np.dot(X, W) + B

행렬의 곱은 대응하는 차원의 수를 일치시켜야함

배치용 Affine 계층

배치용 Affine은 배치 데이터를 Affine하는 것. 

In [7]:
X_dot_W = np.array([[0,0,0], [10,10,10]])
B = np.array([1,2,3])

In [8]:
X_dot_W

array([[ 0,  0,  0],
       [10, 10, 10]])

In [9]:
X_dot_W + B

array([[ 1,  2,  3],
       [11, 12, 13]])

In [10]:
dY = np.array([[1,2,3], [4,5,6]])
dY

array([[1, 2, 3],
       [4, 5, 6]])

In [11]:
dB = np.sum(dY, axis=0)#행을 기준으로 더해준다~ 이말이거든
dB

array([5, 7, 9])

In [12]:
class Affine:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
    
    def forward(self, x):
        self.x = x
        out = np.dot(x,self.W) +self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)#여기서 T는 전치행렬을 만들어주는 역할을함
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

## Softmax-with-loss

출력층에서 사용하는 소프트맥스~

신경망에서 수행하는 작업은 '학습'과 '추론' 두 가지.  
일반적으로 추론할 때는 Softmax 계층을 사용하지 않는다.  
신경망은 추론할 때는 마지막 Affine 계층의 ㅊ출력을 인식 결과로 이용함.  
또한, 신경망에서 정규화하지 않는 출력(softmax 앞의 Affine층을 출력)을 '점수'라고 함.  
즉, 추론에서는 답을 하나만 내는 경우에는 가장 높은 점수만 알면 되니 softmax가 필요 없으나 학습 때는 필요함

In [13]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

역전파 때는 전파하는 값을 배치의 수로 나눠서 데이터 1개당 오차를 앞 계층으로 전파하는 점 주의

# 오차역전파법을 적용한 신경망 구현

In [15]:
import sys, os
sys.path.append(os.pardir)
import numpy as np
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

class TwoLayerNet:#2층 구조
    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std*np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std*np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = OrderedDict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layersw['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        self.lastLayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)
        return x
    
    def loss(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim != 1: t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads

여기서 OrderedDict는 딕셔너리에 추가한 순서를 기억하는 딕셔너리  
이것으로 forward와 backward가 편해짐

## 오차 역전파법으로 구한 기울기 검증하기

수치미분은 오차역전파법이 제대로 됐는지 검증하는 '기울기 확인'에 사용한다.

In [17]:
from dataset.mnist import load_mnist

In [21]:
from two_layer_net import TwoLayerNet

(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label= True)
network = TwoLayerNet(input_size=784, hidden_size= 50, output_size = 10)

x_batch = x_train[:3]
t_batch = t_train[:3]

grad_numerical = network.numerical_gradient(x_batch, t_batch)
grad_backprop = network.gradient(x_batch, t_batch)

for key in grad_numerical.keys():
    diff = np.average(np.abs(grad_backprop[key]-grad_numerical[key]))
    print(key + ":" + str(diff))


W1:4.3558019149151917e-10
b1:2.732961564234568e-09
W2:5.115128229397296e-09
b2:1.3945341181265115e-07


## 오차역전파법을 사용한 학습 구현

In [24]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label= True)
network = TwoLayerNet(input_size=784, hidden_size= 50, output_size= 10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size= x_train.shape[0]
batch_size =100
learning_rate = 0.1

train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)
for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    
    grad = network.gradient(x_batch, t_batch)#오차전역파법으로 기울기 구하기
    
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate*grad[key]
        
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    if i%iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.13348333333333334 0.1424
0.9011166666666667 0.9048
0.9250333333333334 0.9282
0.9337166666666666 0.9347
0.9432 0.9429
0.94995 0.9472
0.95495 0.9503
0.9601166666666666 0.9548
0.9618 0.9563
0.9652833333333334 0.9587
0.9678166666666667 0.9616
0.97065 0.9635
0.97225 0.965
0.9731166666666666 0.9651
0.9748666666666667 0.9679
0.976 0.9676
0.9781333333333333 0.9682
